# Create Annual Calibrated Composite Images for Each RTS Polygon
### To Do:
- get code to iterate over years automatically
- make videos to illustrate composite images with increasing numbers of images for each polygon id and year?

## Set Up Environment

In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap
import os
from pprint import pprint
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
import xarray as xr
import rioxarray as rxr
from datetime import datetime
from collections import Counter
import re

## Import Data and Prepare Visualization Parameters

In [ ]:
# Import Data
yamal = ee.ImageCollection('projects/abruptthawmapping/assets/yg_polygons_imagery_calibrated')
polygons = ee.FeatureCollection('users/gfiske/FrostCraters/rts_polygons_for_Yili_Apr_2022').filterMetadata('code_id', 'equals', 1)

# Prep Map
Map = geemap.Map()
Map.centerObject(yamal)

In [ ]:
# View the imagery
vis_params_imagery = {
    'min': [470, 415, 280],'max': [1180, 930, 750],
    'bands': ['b3', 'b2', 'b1'],
    'gamma': 0.9
}

## Prepare Data

In [ ]:
# Mask the data to values greater than 0 (which is how nodata gets imported into GEE by default)
def mask_0(image):
    mask = image.gt(0)
    return image.updateMask(mask)

yamal = yamal.map(mask_0)

In [ ]:
# Set Properties to allow filtering
def setID(image):
    img_id = image.id();
    img_prop = image.setMulti({'ID': img_id});
    img_prop = ee.Image(img_prop);
    return img_prop;


yamal = yamal.map(setID);

In [ ]:
# check that all of the files got imported into EE
cal_files = os.listdir('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/automated_download/mad_output/calibrated_files/')
cal_files = sorted(cal_files)
cal_files = [filepath[0:-4] for filepath in cal_files]
len(cal_files)
ee_files = yamal.aggregate_array('ID').getInfo()
len(ee_files)
# output from this should be empty
[file for file in cal_files if file not in ee_files]

In [ ]:
# Create list of polygon ID and year combinations
polygon_ids = list(range(0, 68))
polygon_ids_rep = polygon_ids*4
years = np.repeat(list(range(2018, 2022)), 68)
names = [str(pid) + '_' + str(year) for pid, year in zip (polygon_ids_rep, years)]
id_year_combos = ee.List(names) # names[0:106] is the most images I can include without blowing a User Memory Limit Exceeded error
# pprint(id_year_combos.getInfo())

## Create Annual Composites

In [ ]:
# create a composite image for each polygon ID and year
# I can't figure out a way to do this without splitting up the years manually, because I keep getting
# User Memory Limit Exceeded errors
def create_composite_by_group(current, previous):
    return (ee.List(previous)
            .add(yamal.filter(ee.Filter.stringStartsWith('ID', ee.String(current)))
                 .median())
           )

planet_composites_2018 = ee.List(
    ee.List(names[0:68])
    .iterate(create_composite_by_group, ee.List([]))
)
planet_composites_2019 = ee.List(
    ee.List(names[68:136])
    .iterate(create_composite_by_group, ee.List([]))
)
planet_composites_2020 = ee.List(
    ee.List(names[136:204])
    .iterate(create_composite_by_group, ee.List([]))
)
planet_composites_2021 = ee.List(
    ee.List(names[204:272])
    .iterate(create_composite_by_group, ee.List([]))
)

## Map Composites

In [ ]:
# Add composites to the map as one layer
Map.addLayer(ee.ImageCollection(planet_composites_2018),
             vis_params_imagery,
             '2018 Composites')
Map.addLayer(ee.ImageCollection(planet_composites_2019),
             vis_params_imagery,
             '2019 Composites')
Map.addLayer(ee.ImageCollection(planet_composites_2020),
             vis_params_imagery,
             '2020 Composites')
Map.addLayer(ee.ImageCollection(planet_composites_2021),
             vis_params_imagery,
             '2021 Composites')

In [ ]:
Map.addLayer(polygons, {'color': 'FF0000'}, 'RTS');

In [ ]:
Map

## Export Annual Composites

In [ ]:
# Import shapefile with AOI (multipolygon)
aoi = gpd.read_file("/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/automated_download/rts_buffer/RTS_buffer.shp")
# convert from multipolygon to multiple polygons
aoi = aoi.explode(column = 'geometry', ignore_index = True)
# remove inner holes
aoi.geometry = aoi.geometry.exterior
# convert back to polygon
aoi.geometry = [shp.geometry.Polygon([shp.geometry.Point(x, y) for x, y in list(feature.coords)]) for feature in aoi.geometry]
# convert to json for planet data search
sites = json.loads(aoi.to_json()) # if multiple sites

In [ ]:
# # Export Composites to Drive
# for pid in polygon_ids:
#     print(datetime.now().time(), ' Getting info for pid ' + str(pid) + ', year 2018')
#     image = ee.Image(planet_composites_2018.get(ee.Number(int(pid))))
#     name = 'planet_composite_' + str(pid) + '_2018'
#     geometry = sites['features'][pid]['geometry']['coordinates']
#     scale = 3
#     crs = (Counter([band['crs'] for image in (yamal
#                                              .filter(ee.Filter.stringStartsWith('ID', ee.String(names[pid])))
#                                              .getInfo()['features']) for band in image['bands']])
#            .most_common(1)[0][0])
#     print(datetime.now().time(), ' Beginning export for pid ' + str(pid) + ', year 2018')
#     task = ee.batch.Export.image.toCloudStorage(
#         image = image,
#         description = name,
#         bucket = 'abrupt_thaw',
#         fileNamePrefix = 'yamal_gydan_polygons/calibrated_composites/' + name,
#         crs = crs,
#         region = geometry,
#         scale = scale,
#         maxPixels = 1e13,
#         fileFormat = 'GeoTIFF'
#     )
#     task.start()

# for pid in polygon_ids:
#     print(datetime.now().time(), ' Getting info for pid ' + str(pid) + ', year 2019')
#     image = ee.Image(planet_composites_2019.get(ee.Number(int(pid))))
#     name = 'planet_composite_' + str(pid) + '_2019'
#     geometry = sites['features'][pid]['geometry']['coordinates']
#     scale = 3
#     crs = (Counter([band['crs'] for image in (yamal
#                                              .filter(ee.Filter.stringStartsWith('ID', ee.String(names[68 + pid])))
#                                              .getInfo()['features']) for band in image['bands']])
#            .most_common(1)[0][0])
#     print(datetime.now().time(), ' Beginning export for pid ' + str(pid) + ', year 2019')
#     task = ee.batch.Export.image.toCloudStorage(
#         image = image,
#         description = name,
#         bucket = 'abrupt_thaw',
#         fileNamePrefix = 'yamal_gydan_polygons/calibrated_composites/' + name,
#         crs = crs,
#         region = geometry,
#         scale = scale,
#         maxPixels = 1e13,
#         fileFormat = 'GeoTIFF'
#     )
#     task.start()

# for pid in polygon_ids:
#     print(datetime.now().time(), ' Getting info for pid ' + str(pid) + ', year 2020')
#     image = ee.Image(planet_composites_2020.get(ee.Number(int(pid))))
#     name = 'planet_composite_' + str(pid) + '_2020'
#     geometry = sites['features'][pid]['geometry']['coordinates']
#     scale = 3
#     crs = (Counter([band['crs'] for image in (yamal
#                                              .filter(ee.Filter.stringStartsWith('ID', ee.String(names[68*2 + pid])))
#                                              .getInfo()['features']) for band in image['bands']])
#            .most_common(1)[0][0])
#     print(datetime.now().time(), ' Beginning export for pid ' + str(pid) + ', year 2020')
#     task = ee.batch.Export.image.toCloudStorage(
#         image = image,
#         description = name,
#         bucket = 'abrupt_thaw',
#         fileNamePrefix = 'yamal_gydan_polygons/calibrated_composites/' + name,
#         crs = crs,
#         region = geometry,
#         scale = scale,
#         maxPixels = 1e13,
#         fileFormat = 'GeoTIFF'
#     )
#     task.start()

# for pid in polygon_ids:
#     print(datetime.now().time(), ' Getting info for pid ' + str(pid) + ', year 2021')
#     image = ee.Image(planet_composites_2021.get(ee.Number(int(pid))))
#     name = 'planet_composite_' + str(pid) + '_2021'
#     geometry = sites['features'][pid]['geometry']['coordinates']
#     scale = 3
#     crs = (Counter([band['crs'] for image in (yamal
#                                              .filter(ee.Filter.stringStartsWith('ID', ee.String(names[68*3 + pid])))
#                                              .getInfo()['features']) for band in image['bands']])
#            .most_common(1)[0][0])
#     print(datetime.now().time(), ' Beginning export for pid ' + str(pid) + ', year 2021')
#     task = ee.batch.Export.image.toCloudStorage(
#         image = image,
#         description = name,
#         bucket = 'abrupt_thaw',
#         fileNamePrefix = 'yamal_gydan_polygons/calibrated_composites/' + name,
#         crs = crs,
#         region = geometry,
#         scale = scale,
#         maxPixels = 1e13,
#         fileFormat = 'GeoTIFF'
#     )
#     task.start()


## Try Fewer Images in Composites

In [ ]:
# load in file with image order
image_order = pd.read_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/automated_download/planet_images_filtered.csv')
ids_0_2018 = list(image_order[(image_order.polygon_id == 0) & (image_order.year == 2018)].id)
ids_0_2018 = ['0_' + str(x) + '_3B_AnalyticMS_SR_clip_arosics_mad' for x in ids_0_2018][0:-1]
pprint(ids_0_2018)
ids_0_2018 = ee.List(ids_0_2018)

In [ ]:
# try removing images to figure out how many images are required for an image
# very minimal difference with -1 and -2
# noticeable difference, but not necessarily worse at -3, -4, -5, -6
# image noticeably grainier at -7, -8 (= 5 and 4 images total, uncertain of coverage of each image)
# pprint(yamal.getInfo())
test = yamal.filter(ee.Filter.inList('ID', ids_0_2018))
test.getInfo()
test = test.median()
Map.addLayer(test, vis_params_imagery, '0_2018 -1')